# Day 14
## Part 1


In [3]:
import parse
from collections import namedtuple
from dataclasses import dataclass


@dataclass(eq=True, frozen=True)
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)


def parse_data(s):
    data = []
    for line in s.strip().splitlines():
        r = parse.parse("p={px:d},{py:d} v={vx:d},{vy:d}", line)
        data.append((Point(r["px"], r["py"]), Point(r["vx"], r["vy"])))
    return data


def part_1(data, width=101, height=103):
    points = [
        (p + v * 100) % Point(width, height)
        for p, v in data
    ]
    mid_x = width // 2
    mid_y = height // 2
    q1 = sum(1 for p in points if p.x < mid_x and p.y < mid_y)
    q2 = sum(1 for p in points if p.x < mid_x and p.y > mid_y)
    q3 = sum(1 for p in points if p.x > mid_x and p.y < mid_y)
    q4 = sum(1 for p in points if p.x > mid_x and p.y > mid_y)
    return q1 * q2 * q3 * q4


test_data = parse_data("""p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3""")

assert part_1(test_data, 11, 7) == 12

12

In [4]:
data = parse_data(open("input").read())
part_1(data)

225521010

## Part 2

Ha, nice. We're looking for a picture of a Christmas tree which I'm assuming means the robots will be clumped together more than usual. Let's have a look at the standard deviation of x coordinates, finding an outlier there will hopefully give the time the tree shows up.

In [13]:
import statistics
import math

def sd_x(data):
    n = 0
    while True:
        n += 1
        points = [
            (p + v * n) % Point(101, 103)
            for p, v in data
        ]
        yield (statistics.stdev(p.x for p in points), n)

min_sd = math.inf
for (sd, n), _ in zip(sd_x(data), range(100000)):
    if sd < min_sd:
        print(n, sd)
        min_sd = sd

1 28.812932411476215
3 28.679621779424178
7 28.3676902779096
8 27.97283222495303
60 27.906951820645688
98 17.605174476178114


In [18]:
def draw_tree(data, n):
    points = {
        (p + v * n) % Point(101, 103)
        for p, v in data
        }
    return "\n".join(
        "".join("#" if Point(x, y) in points else "." for x in range(103))
        for y in range(101)
    )

print(draw_tree(data, 98))      

................#..........................................#.............#......#......................
.............................................#...#..................#..#.......#.......................
....................................................#....#.........##.....#....#.......................
.................#.................................#......#...#........................................
.........................................................#....#..#.....................................
.........................#..#................#.........#.......#.#.....................................
............................................................#....#.#...........#.......................
..................................................................#.............#...............#......
.........................................................#..#.......#.......#..........................
........................................................#...#...

Hmm, that doesn't look like a tree. Instead calculate the average distance (Manhattan for ease) between points.

In [24]:
import itertools

def manhattan(p1, p2):
    return abs(p1.x - p2.x) + abs(p1.y - p2.y)

def mean_distance(data):
    n = 0
    while True:
        n += 1
        points = {
            (p + v * n) % Point(101, 103)
            for p, v in data
        }
        yield (
            statistics.mean(
                manhattan(p, q) for p, q in itertools.combinations(points, 2)
            ),
            n
        )

min_mean = math.inf
for (mean, n), _ in zip(mean_distance(data), range(10000)):
    if mean < min_mean:
        print(n, mean)
        min_mean = mean

1 67.26730698313722
8 67.04732532524211
18 66.98937426210153
21 65.94024689305458
49 54.87492385423012
98 53.509514658782955
199 52.93909623107958
300 52.77577910391209
1007 52.61181696105774
2522 52.51153133110864
2724 52.206449690921275
5249 51.694816294217134
7774 38.738805611222446


In [26]:
print(draw_tree(data, 7774))

...............................#...........#......................................#....................
...........................................#.......................#...................................
.......................................................................................................
........................................#..............................................................
.......................................................................................................
..............................................#................................#.......................
.......................................................................................................
..........................................#......................................#.....................
...................................................................#.....................#.............
..............#.................................................

That took a long time to run, here's a quicker version that calculates the product of the s.d. of x and y values.

In [27]:
def sd_xy(data):
    n = 0
    while True:
        n += 1
        points = [
            (p + v * n) % Point(101, 103)
            for p, v in data
        ]
        yield (
            statistics.stdev(p.x for p in points) * statistics.stdev(p.y for p in points), 
            n)

min_sd = math.inf
for (sd, n), _ in zip(sd_xy(data), range(10000)):
    if sd < min_sd:
        print(n, sd)
        min_sd = sd

1 847.5422234692347
7 843.5296985811123
8 837.1583029746719
21 809.6083916460806
49 571.8077005287755
98 522.1954446840947
199 522.0276288150632
300 508.83271874869246
2522 508.3831050765549
2724 505.9771249082599
5249 498.41246175503113
7774 336.9559773193042
